# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [4]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [5]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [6]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [8]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [9]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

*********** My Solution Starts Here **********

Step 1: Join CITED_STATE with CITATIONS table as intermediate_table_1

In [10]:
condition_cited = [citations.CITED == patents.PATENT, patents.COUNTRY == 'US']
intermediate_table_1 = citations.join(patents,condition_cited ,how = "inner").select('CITING','CITED',col("POSTATE").alias("CITED_STATE"))

In [11]:
intermediate_table_1.show(2)

+-------+-------+-----------+
| CITING|  CITED|CITED_STATE|
+-------+-------+-----------+
|4133055|3070803|         IL|
|4253313|3070803|         IL|
+-------+-------+-----------+
only showing top 2 rows



Step 2: Join CITING_STATE with the above intermediate_table_1 so that we have CITING_STATE, CITED_STATE together in one table

In [12]:
condition_citing = [citations.CITING == patents.PATENT, patents.COUNTRY == 'US']
intermediate_table_2= intermediate_table_1.join(patents, condition_citing, how = "inner").select('CITING',col("POSTATE").alias("CITING_STATE"),'CITED','CITED_STATE')

In [13]:
intermediate_table_2.show(2)

+-------+------------+-------+-----------+
| CITING|CITING_STATE|  CITED|CITED_STATE|
+-------+------------+-------+-----------+
|3858242|          MI|3319261|         OH|
|3858242|          MI|3707004|         WI|
+-------+------------+-------+-----------+
only showing top 2 rows



Step 3: Filter the latest table (intermediate_table_2) to have citations of the same state only (CITED_STATE == CITING_STATE)

In [15]:
citing_table=intermediate_table_2.filter(intermediate_table_2.CITING_STATE==intermediate_table_2.CITED_STATE)

In [20]:
citing_table.show(5)

+-------+------------+-------+-----------+
| CITING|CITING_STATE|  CITED|CITED_STATE|
+-------+------------+-------+-----------+
|3869295|          AK|3764357|         AK|
|3908753|          AK|3648767|         AK|
|4067198|          AK|3217791|         AK|
|4067198|          AK|3706204|         AK|
|4178878|          AK|3464385|         AK|
+-------+------------+-------+-----------+
only showing top 5 rows



Step 4: Construct the final table with the count of same state citation

In [21]:
final_table=patents.join(citing_table, patents.PATENT==citing_table.CITING, how = "inner").groupBy("PATENT").count().orderBy(col('count').desc())

In [22]:
final_table.show(5)

+-------+-----+
| PATENT|count|
+-------+-----+
|5959466|  125|
|5983822|  103|
|6008204|  100|
|5952345|   98|
|5958954|   96|
+-------+-----+
only showing top 5 rows



Step 5: Join the final table with patent table, group records by patents and count the same state for each patent using group by, order by and count.

In [37]:
result = patents.join(final_table, ['PATENT']).orderBy('count',ascending = False)


Step 6: Arrange the output so taht it matches with the expected final output.

In [38]:
result = result.orderBy(['count', patents.PATENT],ascending = [0,1])
result.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|  125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|  103|
|6008204| 1999|14606|   1998|     US|     CA|  749584| 